In [44]:
from sklearn import tree #decision tree
from sklearn.ensemble import RandomForestRegressor #random Forest
from sklearn.model_selection import cross_val_score # evaluate our model
from sklearn.model_selection import GridSearchCV # gridsearch to find best hyper parameters
import pandas as pd
import numpy as np
import datetime as dt

In [45]:
schedule_csv = '2020_2021_Schedule_updated.csv'
stats_csv = '2020_2021_stats_updated.csv'

In [46]:
### Data Exploration
np.random.seed(1)
df = pd.read_csv(stats_csv, header=[0,1])
df.head(2)

,Unnamed: 0_level_0,Unnamed: 1_level_0,Overall,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Conf.,...,Unnamed: 28_level_0,Unnamed: 29_level_0,Unnamed: 30_level_0,Unnamed: 31_level_0,Unnamed: 32_level_0,Unnamed: 33_level_0,Unnamed: 34_level_0,Unnamed: 35_level_0,Unnamed: 36_level_0,Unnamed: 37_level_0
,Rk,School,G,W,L,W-L%,SRS,SOS,Unnamed: 8_level_1,W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,20,17,3,0.850,5.93,-7.32,NaN,9,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0
1,2,Air Force,19,4,15,0.211,-12.72,0.17,NaN,2,...,184.0,264.0,0.697,78.0,442.0,240.0,127.0,39.0,277.0,307.0


In [47]:
### Data Cleaning

### Only want a single header
df = pd.read_csv(stats_csv, header=[1])
df.head(2)

,Rk,School,G,W,L,W-L%,SRS,SOS,Unnamed: 8,W.1,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,20,17,3,0.850,5.93,-7.32,NaN,9,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0
1,2,Air Force,19,4,15,0.211,-12.72,0.17,NaN,2,...,184.0,264.0,0.697,78.0,442.0,240.0,127.0,39.0,277.0,307.0


In [48]:
#rename w.1 etc
df.rename(columns={"W.1": "Conf W", "W.2": "Home W", "W.3": "Away W"}, inplace=True)
df.rename(columns={"L.1": "Conf L", "L.2": "Home L", "L.3": "Away L"}, inplace=True)
df.head(1)

,Rk,School,G,W,L,W-L%,SRS,SOS,Unnamed: 8,Conf W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,1,Abilene Christian,20,17,3,0.85,5.93,-7.32,NaN,9,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0


In [49]:
df.drop(['Unnamed: 8', 'Unnamed: 11','Unnamed: 17','Unnamed: 20','Unnamed: 14','Rk'], axis=1, inplace = True)
df.head(4)

,School,G,W,L,W-L%,SRS,SOS,Conf W,Conf L,Home W,...,FT,FTA,FT%,ORB,TRB,AST,STL,BLK,TOV,PF
0,Abilene Christian,20,17,3,0.850,5.93,-7.32,9,1,10,...,276.0,428.0,0.645,237.0,718.0,352.0,199.0,63.0,272.0,388.0
1,Air Force,19,4,15,0.211,-12.72,0.17,2,13,2,...,184.0,264.0,0.697,78.0,442.0,240.0,127.0,39.0,277.0,307.0
2,Akron,15,11,4,0.733,5.79,-1.13,9,3,8,...,194.0,260.0,0.746,160.0,592.0,215.0,75.0,41.0,189.0,270.0
3,Alabama A&M,8,6,2,0.750,-13.23,-15.60,4,2,5,...,111.0,172.0,0.645,96.0,329.0,109.0,51.0,30.0,116.0,149.0


In [50]:
df.columns

Index(['School', 'G', 'W', 'L', 'W-L%', 'SRS', 'SOS', 'Conf W', 'Conf L',
       'Home W', 'Home L', 'Away W', 'Away L', 'Tm.', 'Opp.', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF'],
      dtype='object')

In [51]:
### Data Exploration of new table
schedule_df_full = pd.read_csv(schedule_csv)
schedule_df_full.drop('Unnamed: 0',axis=1, inplace=True)
schedule_df_full.tail()

,Date,visitor_neutral,PTS,home_neutral,PTS.1
3797,2021-03-02,Tulsa,0,UCF,0
3798,2021-03-02,Vanderbilt,0,LSU,0
3799,2021-03-02,Wake Forest,0,Pitt,0
3800,2021-03-02,Western Michigan,0,Northern Illinois,0
3801,2021-03-02,Xavier,0,Georgetown,0


In [52]:
schedule_df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3802 entries, 0 to 3801
Data columns (total 5 columns):
Date               3802 non-null object
visitor_neutral    3802 non-null object
PTS                3802 non-null int64
home_neutral       3802 non-null object
PTS.1              3802 non-null int64
dtypes: int64(2), object(3)
memory usage: 148.6+ KB


In [53]:
schedule_df_full.tail()

,Date,visitor_neutral,PTS,home_neutral,PTS.1
3797,2021-03-02,Tulsa,0,UCF,0
3798,2021-03-02,Vanderbilt,0,LSU,0
3799,2021-03-02,Wake Forest,0,Pitt,0
3800,2021-03-02,Western Michigan,0,Northern Illinois,0
3801,2021-03-02,Xavier,0,Georgetown,0


In [54]:
#make school names match
df["School"] = df["School"].replace(["Illinois-Chicago"], "UIC")
df["School"] = df["School"].replace(["Central Florida"], "UCF")

In [55]:
## separate played and upcoming games
schedule_df_played = schedule_df_full[schedule_df_full['PTS']!=0]
schedule_df_unplayed = schedule_df_full[schedule_df_full['PTS']==0]

In [56]:
## Combine schedule with team statistics


In [57]:
combine_played = pd.merge(schedule_df_played,df, left_on='visitor_neutral', right_on='School')
combine_unplayed = pd.merge(schedule_df_unplayed,df, left_on='visitor_neutral', right_on='School')


In [58]:
Schedule_and_stats_played = pd.merge(combine_played, df, left_on = 'home_neutral', right_on = 'School')
Schedule_and_stats_unplayed = pd.merge(combine_unplayed, df, left_on = 'home_neutral', right_on = 'School')


In [59]:
Schedule_and_stats_played["Date"] = pd.to_datetime(Schedule_and_stats_played["Date"])
Schedule_and_stats_unplayed["Date"] = pd.to_datetime(Schedule_and_stats_unplayed["Date"])


In [60]:
Schedule_and_stats_played.to_csv("2020_2021_Played_Games.csv")
Schedule_and_stats_unplayed.to_csv("2020_2021_Unplayed_Games.csv")